In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import glob
import errno
from os.path import isfile
import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
import math

In [2]:
from docx import Document
from docx.shared import RGBColor
from docx import Document
from docx.text.run import Font, Run
import docx

In [3]:
#...........import Bangla stopword................
df = pd.read_excel('bng_sw.xlsx', sheet_name=0)
bng_stopword = df['Stopwords'].tolist()

In [4]:
def remove_stopword(sentence_array):
    for i in range(len(sentence_array)):
        text_tokens = word_tokenize(sentence_array[i])
        tokens_without_sw = [word for word in text_tokens if not word in bng_stopword]
        filtered_text = (" ").join(tokens_without_sw)
        sentence_array[i] = filtered_text

#     print("After Remove Stop-Word:\n", np_sent1)
    return(sentence_array)

In [5]:
def count_actual_token(sentence_array):
    actual_tokens = []
    for i in range(len(sentence_array)):
        text_tokens = word_tokenize(sentence_array[i])
        for j in range(len(text_tokens)):
            actual_tokens.append(text_tokens[j])

    actual_tokens = list(dict.fromkeys(actual_tokens))
    return actual_tokens

In [6]:
def countX(lst, x): 
    count = 0
    for ele in lst: 
        if (ele == x): 
            count = count + 1
    return count 

In [7]:
def count_TF_score(sent_array,token):
    rows, cols = (len(sent_array), len(token)) 
    tf_score = [[0 for i in range(cols)] for j in range(rows)]

    for i in range(rows):
        text_tokens = word_tokenize(sent_array[i])
        count = 0
        total_term = len(text_tokens)
        for j in range(cols):
            rep_term = countX(text_tokens,token[j])
            count = (rep_term/total_term);
            formatted_float = "{:.5f}".format(count)
            tf_score[i][j] = formatted_float
            
    return tf_score

In [8]:
def count_IDF_score(sent_array,token):
    idf_score = []

    for i in range(len(token)):
        count_idf = 0
        score = 0
        for j in range(len(sent_array)):
            sentence = word_tokenize(sent_array[j])
            term = token[i]
            for ele in sentence:
                if (ele == term):
                    count_idf = count_idf + 1
                    break
        score = math.log(len(sent_array)/count_idf,10)
        formatted_float = "{:.5f}".format(score)
        idf_score.append(formatted_float)
    
    return idf_score

In [9]:
def count_TF_IDF_score(np_sentences,token_list):
    rows, cols = (len(np_sentences), len(token_list)) 
    tf_idf_score = [[0 for i in range(cols)] for j in range(rows)]
    
    #........Count TF score of each sentence..............
    tf_score = count_TF_score(np_sentences,token_list)
    
    #........Count IDF score of each sentence..............
    idf_score = count_IDF_score(np_sentences,token_list)

    for i in range (rows):
        for j in range(cols):
            tf = tf_score[i][j]
            idf = idf_score[j]
            score = float(tf) * float(idf)
            formatted_float = "{:.5f}".format(score)
            tf_idf_score[i][j] = formatted_float
           
    return tf_idf_score

In [10]:
#...........Read Test Data..................
f = open("Test_data_1.txt", encoding = 'utf-8')
a2 = f.read()
# print("Test Data Is: ",text)

# .........split every sentence & store an array............
sentence2 = a2.split("।")
np_sent2 = np.array(sentence2)

# .........remove last row for blank line...........
np_sent2 = np.delete(np_sent2,-1)

#.........Store original sentences in an array.......
org_sent = []
for sentence in np_sent2:
    org_sent.append(sentence)
    
org_sent = np.insert(org_sent,0,'Test Article Is: ')
print(org_sent)

#........Mark orginal sentence index...........
mark_org_index = np.zeros(len(org_sent))

#.....create a array for mark text sent is it copy....
mark_line = np.zeros(len(np_sent2))
# print(mark_line)

path = 'C:/Users/DOLPHIN/Downloads/Plagiarism Detection V-4.3/Train_Data'

files=filter(isfile,glob.glob('%s/*'%path))

#.........Declear some variable..........
book_name = []
plagiarism_percent = []
copy_line = []

for name in files:
        f = open(name, encoding = 'utf-8')
        name = str(name).replace('C:/Users/DOLPHIN/Downloads/Plagiarism Detection V-4.3/Train_Data', '')
        name=name.replace("\\","")
#         print("\n Train Doc iS: ",name)
        book_name.append(name)
        a1 = f.read()
#         print(a1)
        f.close()
        
        # ......split every sentence & store an array
        sentence1 = a1.split("।")
        sentence2 = a2.split("।")

        # ......store each document in different array
        np_sent1 = np.array(sentence1)
        np_sent2 = np.array(sentence2)

        # ......remove last row for blank line
        len_np_sent1 = len(np_sent1)-1
        len_np_sent2 = len(np_sent2)-1

        #print(len_np_sent1,len_np_sent2)
        
        #print("train data:", np_sent1)
        
        np_sent1 = np.delete(np_sent1,len_np_sent1)
        np_sent2 = np.delete(np_sent2,len_np_sent2)

        # .....store values os np_sent2 in np_sent1
        np_sent1 = np.concatenate((np_sent1[:],np_sent2[:]))
    
        #print("mark line:",mark_line)
        #print("\n Total Sentences: ",np_sent1)
        #print(len(np_sent1))
        
        #.........Remove Stop-Word From Each Sentence........
        filtered_sentence = remove_stopword(np_sent1)
        # print(filtered_sentence)

        #........Count actual token of full article..........
        actual_tokens = count_actual_token(filtered_sentence)
        # print(actual_tokens)

        #........Count TF-IDF score of each sentence..........
        tfidf_matrix_train = count_TF_IDF_score(filtered_sentence,actual_tokens)
        #print(tfidf_matrix_train[:]);
        
        #..........run cosine similarity algorithms.........
        flag1 = len_np_sent1
        flag2 = flag1 + 1
        count_copy = 0
        x = 0
#         print(flag1,flag2)
#        print("mark line:",mark_line)
        
        while flag1 != len(np_sent1):
            cosine_array = np.array(cosine_similarity(tfidf_matrix_train[flag1:flag2], tfidf_matrix_train))
            #print("cosine scores ==> ",cosine_array)
            
        #......identify test data sentance index.........
#             print("Testing index")
#             print("falg2",flag2-1,"len1",len_np_sent1-1)
#             mark_index = ((flag2-1)-(len_np_sent1-1))-1
#             print("index is:" , mark_index)
            
            flag1 = flag2
            flag2 = flag1 + 1
            result = cosine_array.flatten()
            result = result[:-len_np_sent2]

            #.........remove test data result.....

#             print("length:",len(result))
#             print("1D is:",result)
            max_value = np.amax(result)
#             print('Max Value from  Array : ', max_value)
            if (max_value >= 0.90 and mark_line[x]== 0):
                mark_line[x] = 1
                x = x+1
                mark_org_index[x] = 1
#                 print("mark line:",mark_line,"\n")
                count_copy = count_copy + 1
            else:
                x = x+1

#         print("No of Lines copied:" , count_copy)
        copy_line.append(count_copy)
        
        #......Calculate  plagiarism in with train doc..........
        total_line = len_np_sent2
        copied_line = count_copy
        plagiarism = (copied_line/total_line)*100
#         print("Plagiarism Result is: ",plagiarism,"%")
        plagiarism_percent.append(plagiarism)
        

['Test Article Is: '
 'অনেকেই ভাবে, যার মস্তিষ্ক যত বড়, তার চিন্তা করার শক্তি তত বেশি'
 ' ধারণাটি কিন্তু ভুল' ' মানুষের থেকে তিমির মস্তিষ্ক অনেক বড়'
 ' কিন্তু তিমির থেকে মানুষ বেশি বুদ্ধিমান'
 ' নেতৃত্ব, জ্ঞান–বিজ্ঞান, সাহিত্যচর্চা—এসব বিষয়ে মানুষ উন্নতি করতে পেরেছে'
 ' বাংলা ব্যঞ্জনসন্ধির সূত্র মুখস্থ করতে করতে অনেকে অস্থির হয়ে যায়'
 ' সমীভবন হলো এমন এক প্রক্রিয়া, যেখানে এক ধ্বনির প্রভাবে পাশের ধ্বনিটি বদলে যায়'
 ' আমার ছোট ফুফুর বাড়ি' ' বহুদিন পর ফুফুর বাড়ি বেড়াতে এলাম'
 ' হাঁটতে ভালো লাগছে' ' মাঠ পেরোনোর আগেই সন্ধ্যা হয়ে গেল'
 ' খোলা জায়গায় অন্ধকার জমতে সময় লাগে'
 ' সূর্য ডোবার পরও অনেকটাক্ষণ আলোকিত ভাব থাকে'
 '  অন্ধকারে ভয় পাওয়ার কিছু নেই' ' সঙ্গে টর্চ নেই, মোবাইল তো আছে'
 ' মোবাইলই এখন টর্চের কাজ দেয়'
 ' ২৪০ পাতার বইটিকে নিছক ভুয়ো বলে মানতে রাজি নন বিশেষজ্ঞরা'
 ' উর্দু ডানদিকে লেখা হয়েছে বইটি'
 ' ভেষজ বা আয়ুর্বেদিক চিকিৎসাবিদ্যার নানা দিক নিয়ে বিস্তৃত আলোচনা রয়েছে বইয়ে যা সত্যিই অবাক করে'
 ' সম্ভবত ১৯১২ সালে বইটি কিনে নেন বইয়ের ব্যবসা করা উইলফ্রিড ভয়নিচ'
 ' তার নামেই বইটির এই নাম

In [11]:
#........calculate total plagiarism of test data...........
total_line = len_np_sent2
copied_line = 0

for x in range(len(mark_line)):
    if mark_line[x] == 1:
        copied_line = copied_line + 1
        
plagiarism = (copied_line/total_line)*100
print("Total Plagiarism Found : ",plagiarism,"%")    

Total Plagiarism Found :  56.52173913043478 %


In [13]:
mydoc = docx.Document()
other_source = 0
line_count = 0

for i in range(len(mark_org_index)):
    if i == 0:
        first = mydoc.add_paragraph(org_sent[i])
        
    elif mark_org_index[i]==0:    
        pera = first.add_run(org_sent[i]+"।")
        
    else:
        pera = first.add_run(org_sent[i]+"।")
        font = pera.font
        font.color.rgb = RGBColor(255, 0, 0)

        
first = mydoc.add_paragraph("Plagiarim Result:")

for book in range(len(book_name)):
    if plagiarism_percent[book]>5.0:
        pera = mydoc.add_paragraph().add_run(book_name[book]+" : "+str(plagiarism_percent[book])+"% || "+str(copy_line[book])+" Lines Copied")
        font = pera.font
        font.color.rgb = RGBColor(0, 51, 102)
    else:
        others_source = other_source + plagiarism_percent[book]
        line_count = line_count + copy_line[book]

pera = mydoc.add_paragraph().add_run("Others Source : "+str(others_source)+"% ||"+str(line_count))
font = pera.font
font.color.rgb = RGBColor(0, 51, 102)
pera = mydoc.add_paragraph().add_run("Total Plagiarism : "+str(plagiarism)+"%")
font = pera.font
font.color.rgb = RGBColor(0, 51, 102)
        
mydoc.save("report.docx") 